In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import numpy as np

----

In [48]:
def get_dummies(df, col_name) : 
    """
    입력된 dataframe에 입력한 column의 더미 변수가 원래 데이터 프레임에 붙어 나오는 함수입니다.
    이번 작업에서는 성별(CLNT_GENDER)의 더미 변수 제작을 위해 사용되었습니다.
    
    """
    will_be_dummy = pd.DataFrame(df[col_name]) 
    dummy_variable = pd.get_dummies(will_be_dummy)
    df = df.drop(columns = [col_name])
    return df.join(dummy_variable)

def make_resampling(df, product_name) : 
    """
    입력한 product_name에 해당하는 상품만 필터링 후, pandas의 resampling을 기능을 통해 
    종속변수(판매 수량)와 독립변수들(판매 수량 이외 변수들)의 데이터 프레임을 재생산하는 함수입니다.
    
    """
    
    df["SESS_DT"] = pd.to_datetime(df["SESS_DT"])
    cond = np.where(df["CLAC3_NM"] == product_name)[0]
    df = df.iloc[cond, : ]
    df = df.set_index(keys = ["SESS_DT"])
    
    #평균으로 다룰 cols
    X_col_names = [col for col in df.columns if col != "PD_BUY_CT" and col != "CLAC3_NM"]
    #sum으로 다룰 cols
    y_col_name = ["PD_BUY_CT"]
    
    left = df[X_col_names].resample('1W', convention = 'start').mean()
    right = df[y_col_name].resample('1W', convention = 'start').sum()
    train = left.join(right)
    if sum(train.isnull().sum()) >= 1 : 
        train = train.fillna(value = 0)
    
    return  train

def get_X_y(train_with_ss) : 
    """
    Standardize 된 train 셋을 받아 독립, 종속변수로 나누는 함수입니다.
    
    """
    X = train_with_ss[[col for col in train_with_ss.columns if col != "PD_BUY_CT"]]
    y = train_with_ss["PD_BUY_CT"]
    return X, y

def split_train_test(X, y) : 
    """
    지정한 비율로 train, test 데이터 셋을 나누는 함수 입니다.
    이번 작업에서는 0.7의 비율로 train, test 데이터 셋을 나누어 사용하였습니다. 
    
    """
    portion = 0.7
    split_length = int(len(X) * portion)
    
    
    X_train = X.iloc[ : split_length, : ]
    X_test = y[ : split_length]
    y_train = X.iloc[split_length : , : ]
    y_test = y[split_length : ]
    
    return X_train, X_test, y_train, y_test  

def ready_data_set(df_with_dummy, product_name) : 
    """
    위 세 함수를 한데 모아, 전체 데이터 셋을 입력하면, 
    학습과 성능 검증에 사용할 네 가지 데이터 셋을 return 합니다. 
    
    """
    
    train = make_resampling(df_with_dummy, product_name)
    ss = StandardScaler()
    train_with_ss = ss.fit_transform(train)
    train_with_ss = pd.DataFrame(train_with_ss, columns = train.columns, index = train.index)
    X, y = get_X_y(train_with_ss)
    X_train, X_test, y_train, y_test = split_train_test(X, y)
    return X_train, X_test, y_train, y_test

def get_exp_multiplier_list(X_test) :
    """
    T(시점상 가장 최신값)을 기준으로 T-1, T-2...T-N에서 N값이 커질 수록 날짜상으로는 과거로 가서
    평행사변형의 넓이를 구합니다. 평행사변형의 넓이는 지수 함수의 승수로 각각 들어가며,
    때문에 함수 이름이 exponential(지수)의 multiplier(승수) 로 사용할 값들의 list를 구한다는 뜻입니다.
    """
    
    X_test_arr = np.array(X_test)    

    area_list = []
    
    for idx in range(-1, -len(X_test_arr), -1) : 
        
        T = X_test_arr[idx]
        T_1 = X_test_arr[idx-1]
        area = np.abs((T + T_1) / 2) #평행사변형의 넓이 구하는 공식
        area_list.append(area)
        
    area_list_arr = np.array(area_list)
    total = np.sum(area_list_arr)
    proported_area_list = area_list_arr / total
    
    exp_multiplier_list = []
    T_1_exp_multiplier = 1 - proported_area_list[0]
    exp_multiplier_list.append(T_1_exp_multiplier)
    
    for idx in range(1, len(proported_area_list)) : 
        T_1_exp_multiplier = exp_multiplier_list[-1] - proported_area_list[idx]
        exp_multiplier_list.append(T_1_exp_multiplier)
    exp_multiplier_list.insert(0, 1)
    return exp_multiplier_list

def get_adjusted_coef_list(coef, exp_multiplier_list) : 
    """
    get_exp_multiplier_list 함수를 통해 얻어진 exponential의 multiplier 리스트로
    이전에 학습한 데이터의 다중선형회귀 방정식을 내적하여 
    
    """
    adjusted_coef_list = []
    for x in exp_multiplier_list : 
        
        adjusted_coef = coef * np.exp(x - 1) # 승수의 최댓값을 0으로 맞추기 위해 지수함수 그래프 이동
        adjusted_coef_list.append(adjusted_coef)
       
    return adjusted_coef_list

def modifying_with_adjust_coef(df, adjusted_coef_list) : 
    """
    df, y_train 서로 길이가 다른 두 데이터 셋이 모두 들어올 수 있도록 만들 것.
    
    """   
    
    adjusted_coef_list = adjusted_coef_list[::-1] #reverse
    
    modified_X_test = []
    for idx in range(len(adjusted_coef_list)) : 
        adjusted_coef = adjusted_coef_list[idx]
        row = df.iloc[idx, : ]  
        predict_value = np.dot(row, adjusted_coef)
        modified_X_test.append(predict_value)
    
    return modified_X_test

def get_result(model, X_train, X_test, y_train) : 
    """
    위 함수들을 조합하여 지수함수를 적용하여 수정 된 예측값을 얻는 함수입니다.
    
    """
    temp_ls = []
    modified_predict = []
    for idx in range(len(y_train)) : 
        # 그래프 넓이를 이용한 지수 함수의 승수 구하기
        exp_multiplier_list = get_exp_multiplier_list(X_test)
        # 지수함수를 적용한 수정 된 계수 구하기
        adjusted_coef_list = get_adjusted_coef_list(model.coef_, exp_multiplier_list)
        # 수정된 계수를 이용하여 최신성을 반영한 y값 생성
        modified_X_test = modifying_with_adjust_coef(X_train, adjusted_coef_list)
        modified_X_test_df = pd.DataFrame(modified_X_test)
        temp_ls.append(modified_X_test_df)
        # 수정 된 y값을 이용한 모델 트레이닝
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train, modified_X_test)
        row = np.array(y_train.iloc[idx, : ]).reshape(-1, 1) # 내적을 위해 행렬 형태 변환
        
        row = row.reshape(1, len(y_train.columns))
        predict_with_adjusted_coef = model.predict(row)[0]
        X_train.loc[y_train.index[idx]] = y_train.iloc[idx, : ]
        modified_X_test.append(predict_with_adjusted_coef)
        modified_predict.append(predict_with_adjusted_coef)
        #X_test 갱신
        X_test = modified_X_test
        
    return modified_X_test, modified_predict, temp_ls

def get_score(y_test, modified_predict, predict) : 
    """
    학습 모델의 일반적인 예측값과, 지수함수가 적용되어 수정된 예측값의 차이를
    보기 위해 만든 함수입니다.
    
    """
    modified_result = mean_squared_error(y_test, modified_predict)
    result = mean_squared_error(y_test, predict)
    return result, modified_result

def analysis_using_exp(df_with_dummy) :     
    """
    전처리 된 데이터를 입력하면, 데이터 분리부터 예측값까지 한꺼번에 수행되어
    최종 결과를 받아볼 수 있는 함수입니다.
    
    """
    
    product_name_list = df_with_dummy["CLAC3_NM"].unique()
    
    grade_card = pd.DataFrame(columns = ["PRODUCT_NAME", "BEFORE", "AFTER"])

    cnt = 0
     
    for product_name in product_name_list : 
        try : 
            cnt += 1
            if cnt % 100 == 0 :
                print("{}번째 완료".format(cnt))


            X_train, X_test, y_train, y_test = ready_data_set(df_with_dummy, product_name)
            model = LinearRegression(fit_intercept=False)
            model.fit(X_train, X_test)
            predict = model.predict(y_train)
            modified_X_test, modified_predict, temp_ls = get_result(model, X_train, X_test, y_train)
            result, modified_result = get_score(y_test, modified_predict, predict)
            data = {
                "PRODUCT_NAME" : product_name,
                "BEFORE" : result,
                "AFTER" : modified_result,
            }
            grade_card.loc[len(grade_card)] = data
        except : 
            print("{} 오류 발생 ".format(product_name))
    return grade_card, temp_ls

In [31]:
df = pd.read_csv("./AfterPreprocessing.csv", engine = "python", encoding="utf-8", parse_dates=["SESS_DT"])

In [32]:
df_with_dummy = get_dummies(df, "CLNT_GENDER")

In [49]:
grade_card, temp_ls = analysis_using_exp(df)

100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
감 오류 발생 
아이젠 오류 발생 
볶음반찬 오류 발생 
부채 오류 발생 
과실주병 오류 발생 
식기건조기 오류 발생 
살구 오류 발생 
반찬세트 오류 발생 
발 오류 발생 
로만쉐이드/벌룬쉐이드 오류 발생 
수도용품 오류 발생 
방한모 오류 발생 
석류 오류 발생 


---

In [70]:
grade_card

,PRODUCT_NAME,BEFORE,AFTER
0,남성스포츠티셔츠,3.271940e+00,1.348267
1,남성스포츠샌들/슬리퍼,7.820067e-01,1.225248
2,아쿠아슈즈,1.768187e+00,1.244986
3,남성티셔츠,1.515197e+00,0.964246
4,영유아티셔츠/탑,3.055134e-01,0.926556
5,남아바지,6.460318e-01,0.519950
6,영유아남방셔츠,1.127476e+01,0.608412
7,남성일반스포츠바지,2.702673e+00,1.561078
8,유아동스니커즈,4.318478e-01,0.996141
9,남성정장셔츠,1.087249e+00,0.921655
